In [35]:
import plotly.express as px
from utils.ops import load_yaml
from  pathlib import Path
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Preparing

In [36]:
site = 2
site_label  = 'AM'

In [37]:
cfg = load_yaml(f'cfg{site}.yaml')
site_cfg = load_yaml(f'site_{site}.yaml')

preparation_params = cfg['preparation_params']
paths_params = cfg['paths']
general_params = cfg['general_params']
experiments_paths = general_params['experiments_folders']

images_path = Path('images')
images_path.mkdir(exist_ok=True)

## Functions

In [38]:
def results2(experiments):
    exp_results = []
    for exp_i in experiments:
        #experiment_params = cfg['experiments'][f'exp_{experiments[exp_i]}']
        exp_path = Path(paths_params['experiments']) / f'exp_{exp_i}'
        experiment_params = cfg['experiments'][f'exp_{exp_i}']

        results_path = exp_path / experiments_paths['results']
        final_results_file = results_path / f'results_{exp_i}.data'

        result_df = pd.read_pickle(final_results_file)

        exp_results.append(
            {
                'name': f'{experiment_params["short"]}', # ({exp_i})',
                'cloud_cond': 'All Pixels',
                'f1': result_df['global_f1'][0],
                'precision': result_df['global_precision'][0],
                'recall': result_df['global_recall'][0],
                'group_model': experiment_params['group_model'],
                'prev_def': 'False' if '[No Prev Def]' in experiment_params['short'] else 'True'
            }
        )

        exp_results.append(
            {
                'name': f'{experiment_params["short"]}', # ({exp_i})',
                'cloud_cond': 'No Cloud',
                'f1': result_df['no_cloud_f1'][0],
                'precision': result_df['no_cloud_precision'][0],
                'recall': result_df['no_cloud_recall'][0],
                'group_model': experiment_params['group_model'],
                'prev_def': 'False' if '[No Prev Def]' in experiment_params['short'] else 'True'
            }
        )

        exp_results.append(
            {
                'name': f'{experiment_params["short"]}', # ({exp_i})',
                'cloud_cond': 'Cloudy',
                'f1': result_df['cloud_f1'][0],
                'precision': result_df['cloud_precision'][0],
                'recall': result_df['cloud_recall'][0],
                'group_model': experiment_params['group_model'],
                'prev_def': 'False' if '[No Prev Def]' in experiment_params['short'] else 'True'
            }
        )


        
    return pd.DataFrame(exp_results)

## Data preparing

In [39]:
experiments = [
    6,
    12,
    2,
    13,
    56,
    62,
    52,
    63
    
]
exp_results = results2(experiments)

In [40]:
exp_results

,name,cloud_cond,f1,precision,recall,group_model,prev_def
0,CNN-OPT (Cloud-free),All Pixels,0.825462,0.907231,0.757214,CNN,True
1,CNN-OPT (Cloud-free),No Cloud,0.827067,0.910950,0.757329,CNN,True
2,CNN-OPT (Cloud-free),Cloudy,0.212470,0.128307,0.617553,CNN,True
3,CNN-OPT [No Prev Def] (Cloud-free),All Pixels,0.681163,0.981681,0.521514,CNN,False
4,CNN-OPT [No Prev Def] (Cloud-free),No Cloud,0.681636,0.981679,0.522069,CNN,False
5,CNN-OPT [No Prev Def] (Cloud-free),Cloudy,0.076445,1.000000,0.039742,CNN,False
6,CNN-SAR,All Pixels,0.839339,0.855540,0.823740,CNN,True
7,CNN-SAR,No Cloud,0.838706,0.857185,0.821008,CNN,True
8,CNN-SAR,Cloudy,0.842192,0.848235,0.836234,CNN,True
9,CNN-SAR [No Prev Def],All Pixels,0.754273,0.849373,0.678325,CNN,False


In [43]:
exp_results.loc[(exp_results['name'].str.contains('SAR') & exp_results['cloud_cond'].str.contains('Cloudy')), 'f1'] = np.nan
exp_results.loc[(exp_results['name'].str.contains('SAR') & exp_results['cloud_cond'].str.contains('No Cloud')), 'f1'] = np.nan

#cnn_exp_results = exp_results[exp_results['group_model'] == 'CNN']
cnn_exp_results = exp_results
cnn_max = cnn_exp_results[cnn_exp_results['name'].str.contains('Cloud-free')]['f1'].max()
cnn_exp_results.loc[(cnn_exp_results['name'].str.contains('Cloud-free') & exp_results['cloud_cond'].str.contains('No Cloud')), 'f1'] = np.nan
cnn_exp_results.loc[(cnn_exp_results['name'].str.contains('Cloud-free') & exp_results['cloud_cond'].str.contains('Cloudy')), 'f1'] = np.nan

x_lim1 = 2/4.

cnn_exp_results = cnn_exp_results.replace(' \(Cloud-free\)', '', regex=True)
cnn_exp_results = cnn_exp_results.replace(' \[No Prev Def\]', '', regex=True)

fig = px.bar(cnn_exp_results, x="name", y="f1",
             color='prev_def', barmode='group',
             labels = {
                 'f1': 'F1-Score',
                 'name': 'Model',
                 'cloud_cond': 'Cloud Condition',
                 'prev_def': 'Previous Deforestation Map'
                 },
             height=400,
             width= 800)

fig.add_hrect(y0=0, y1=1, x0 = 0.0, x1 = x_lim1, line_width=0, fillcolor="green", opacity=0.15, layer = 'below')
fig.add_hrect(y0=0, y1=1, x0 = x_lim1, x1 = 1, line_width=0, fillcolor="purple", opacity=0.15, layer = 'below')
fig.add_annotation(text='CNN Models', x=0.5, y=1.05, showarrow = False)
fig.add_annotation(text='Transformer-based Models', x=2.5, y=1.05, showarrow = False)

fig.update_layout(title_text=f'Previoius Deforestation Map Improvement (Site {site})', title_x=0.5)
fig.show()
fig.write_image(f'images/site_{site}_def_prev.png')